### Hierarchical Indices in Document Retrieval

#### Overview
###### This code implements a Hierarchical Indexing system for document retrieval, utilizing two levels of encoding: document-level summaries and detailed chunks. This approach aims to improve the efficiency and relevance of information retrieval by first identifying relevant document sections through summaries, then drilling down to specific details within those sections.

#### Motivation
###### Traditional flat indexing methods can struggle with large documents or corpus, potentially missing context or returning irrelevant information. Hierarchical indexing addresses this by creating a two-tier search system, allowing for more efficient and context-aware retrieval.

#### Key Components
###### PDF processing and text chunking
###### Asynchronous document summarization using OpenAI's GPT-4
###### Vector store creation for both summaries and detailed chunks using FAISS and OpenAI embeddings
###### Custom hierarchical retrieval function

#### Method Details
###### Document Preprocessing and Encoding
###### The PDF is loaded and split into documents (likely by page).
###### Each document is summarized asynchronously using GPT-4.
###### The original documents are also split into smaller, detailed chunks.
###### Two separate vector stores are created:
###### One for document-level summaries
###### One for detailed chunks
###### Asynchronous Processing and Rate Limiting
###### The code uses asynchronous programming (asyncio) to improve efficiency.
###### Implements batching and exponential backoff to handle API rate limits.

#### Hierarchical Retrieval
###### The retrieve_hierarchical function implements the two-tier search:

###### It first searches the summary vector store to identify relevant document sections.
###### For each relevant summary, it then searches the detailed chunk vector store, filtering by the corresponding page number.
###### This approach ensures that detailed information is retrieved only from the most relevant document sections.

#### Benefits of this Approach
###### Improved Retrieval Efficiency: By first searching summaries, the system can quickly identify relevant document sections without processing all detailed chunks.
###### Better Context Preservation: The hierarchical approach helps maintain the broader context of retrieved information.
###### Scalability: This method is particularly beneficial for large documents or corpus, where flat searching might be inefficient or miss important context.
###### Flexibility: The system allows for adjusting the number of summaries and chunks retrieved, enabling fine-tuning for different use cases.

#### Implementation Details
###### Asynchronous Programming: Utilizes Python's asyncio for efficient I/O operations and API calls.
###### Rate Limit Handling: Implements batching and exponential backoff to manage API rate limits effectively.
###### Persistent Storage: Saves the generated vector stores locally to avoid unnecessary recomputation.

#### Conclusion
###### Hierarchical indexing represents a sophisticated approach to document retrieval, particularly suitable for large or complex document sets. By leveraging both high-level summaries and detailed chunks, it offers a balance between broad context understanding and specific information retrieval. This method has potential applications in various fields requiring efficient and context-aware information retrieval, such as legal document analysis, academic research, or large-scale content management systems.


### Import libraries

In [ ]:
import asyncio
import os
import sys
from dotenv import load_dotenv 
from langchain_openai import ChatOpenAI
from langchain.chains.summarize.chain import Document

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) #Add the parent directory to the path since we work with notebooks
from helper_functions import *
from evaluation.evaluate_rag import *
from helper_functions import encode_pdf, encode_from_string

#Load environment variables from a .env file
load_dotenv()

#Set the OpenAI API key environment variables
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Define Document Path

In [ ]:
path = "../data/Understanding_Climate_Change.pdf"

### Function to encode to both summary & chunk levels, sharing the page metadata

In [ ]:
async def encode_pdf_hierarchical(path, chunk_size=1000, chunk_overlap=200, is_string=False):
   """
   Asynchronously encodes a PDF book into a hierarchical vector store using OpenAI embeddings
   Includes rate limit handling with exponential backoff

   Args:
      path: The path to the PDF file
      chunk_size: The desired size of each text chunk
      chunk_overlap: The amount of overlap between consecutive chunks

   Returns:
      A tuple containing two FAISS vector stores:
      1. Document level summaries
      2. Detailed chunks
   """

   # Load PDF documents
   if not is_string:
      loader = PyPDFLoader(path)
      documents = await ayncio.to_threat(loader.load)
   else:
      text_splitter = Recursive

